In [106]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Landscape slide duplicate\Landscape Replacement Function.ipynb"

In [2]:
start_time = time.time()

## Data Parameters

In [3]:
client_manuf = ["Edgewell"]
client_brands = ["Billie","Hydro Silk","Intuition","Skintimate"]

decimals = 2
sign = "Before"
currency = '$'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '
 

categories = ["Manual Shave Women"]
sectors = ["Dermaplane","Disposable","System"]
segments = ["Dermaplane","Disposable","Hybrid","Razors","Refills"]
subsegments= ["1 Blade", "2 Blade", "3 Blade", "4 Blade", "5 Blade", "6 Blade","Rem Blades"]
subcategories= ["Dry Skin","Normal Skin","Rem Types","Sensitive Skin"]
 
national = True
customareas= ""
areas = ['NATIONAL',"RETAILER"]

regions_RET  = ["Amazon", "CVS Corp", "Walmart"]
channels_RET = ["Amazon 1P", "CVS", "Walmart Div1 Corp", "Walmart Nm Corp", "Walmart Sc Corp","Amazon 3P"]
market_RET = []

regions_CHAN = []
channels_CHAN = []
market_CHAN = []
 
regions_CUST = []
channels_CUST = []
market_CUST = []

data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending August 2024"
 
years = ['2022','2023', "2024"]


ManufOrTopC ="Top Companies"
BrandOrTopB = "Top Brands"

# subcatg_parent = "Segment"
# subcatg_parent_list = segments
percent = 100000
percentstr="'00 000"

Sections_WO_CB=False

In [4]:
retMarketLength = 3-((not regions_RET)+(not channels_RET)+(not market_RET))
chanMarketLength = 3-((not regions_CHAN)+(not channels_CHAN)+(not market_CHAN))
custMarketLength = 3-((not regions_CUST)+(not channels_CUST)+(not market_CUST))

In [5]:
retailers = regions_RET + channels_RET + market_RET
channels = regions_CHAN + channels_CHAN + market_CHAN
cust = regions_CUST + channels_CUST + market_CUST

### Cleaning Data_frames

In [6]:
loaded_data = {}

datasets_path = os.getcwd()+"/Landscape Datasets/"
print(datasets_path)

datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)

c:\Users\RaphaellaASHRAF\OneDrive - Pricing One SA\Documents\GitHub\Slide-Automate\Slide-Automate\Landscape slide duplicate/Landscape Datasets/


In [7]:
def secsegclean(inputdic):
    outputdic={}
    totaloutputdic={}
    for s in inputdic.keys():
        t = inputdic[s].copy()
        t=DetectHeader(t).fillna(0)
        mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
        mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
        tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
        if not mod.empty:
            outputdic[s] = mod
        if not tot.empty:
            totaloutputdic[s] = tot
    return outputdic,totaloutputdic

In [8]:
if len(sectors)!=0:
    modified_sectors_dfs_new = {}
    sectors_totals_new= {}
    modified_sectors_dfs_new, sectors_totals_new = secsegclean(sectors_dfs)
    #P12M
    modified_sectors_P12M_new = {}
    dya_sectors_new = {}
    modified_sectors_P12M_new, dya_sectors_new = secsegclean(sectors_P12M_dfs)
    #Client
    modified_sectors_clients_new = {}
    modified_sectors_clients_new,_ = secsegclean(sectors_client_dfs)

if len(segments)!=0:
    modified_segment_dfs_new = {}
    segment_totals_new= {}
    modified_segment_dfs_new, segment_totals_new = secsegclean(segments_dfs)
    #P12M
    modified_segment_P12M_new = {}
    dya_segment_new = {}
    modified_segment_P12M_new, dya_segment_new = secsegclean(segments_P12M_dfs)
    #Client
    modified_segment_clients_new = {}
    modified_segment_clients_new,_ = secsegclean(segments_client_dfs)


if len(subsegments)!=0:
    modified_subsegment_dfs_new = {}
    subsegment_totals_new= {}
    modified_subsegment_dfs_new, subsegment_totals_new = secsegclean(subsegments_dfs_new)
    #P12M
    modified_subsegment_P12M_new = {}
    dya_subsegment_new = {}
    modified_subsegment_P12M_new, dya_subsegment_new = secsegclean(subsegments_P12M_dfs_new)
    #Client
    modified_subsegment_clients_new = {}
    modified_subsegment_clients_new,_ = secsegclean(subsegments_client_dfs_new)


if len(subcategories)!=0:
    modified_subcategories_dfs_new = {}
    subcategories_totals_new= {}
    modified_subcategories_dfs_new, subcategories_totals_new = secsegclean(subcategories_dfs_new)
    #P12M
    modified_subcategories_P12M_new = {}
    dya_subcategories_new = {}
    modified_subcategories_P12M_new, dya_subcategories_new = secsegclean(subcategories_P12M_dfs_new)
    #Client
    modified_subcategories_clients_new = {}
    modified_subcategories_clients_new,_ = secsegclean(subcategories_client_dfs_new)


In [9]:
def addOther(dic,col=f"{BrandOrTopB}",TorB="Top"):
    for key, df in dic.items():
        if col in df.columns:
            others_row = df[df[col].str.contains('Others', case=False, na=False)]
            rows_wo_others = df[~df[col].str.contains('Others', case=False, na=False)]
            if not others_row.empty:
                if TorB=="Top":
                # Re-order: "Others" first, then the rest of the rows
                    dic[key] = pd.concat([others_row, rows_wo_others])    
                else:
                    dic[key] = pd.concat([ rows_wo_others,others_row])    
    return dic                    

In [10]:
modified_manuf_dfs_new = {}
modified_manuf_totals_new= {}
modified_manuf_P12M_new = {}
modified_manuf_P12M_regions_new = {}

for s in manuf_dfs_new.keys():
    t = manuf_dfs_new[s].copy()
    t=DetectHeader(t).fillna(0)
    t.columns = t.columns.str.strip()            

    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        modified_manuf_dfs_new[s] = mod
    if not tot.empty:
        modified_manuf_totals_new[s] = tot
    

df_toDrop =[key for key in modified_manuf_dfs_new.keys() if (modified_manuf_dfs_new[key].shape[1]>2) and (modified_manuf_dfs_new[key].shape[1]<4)]

for key in df_toDrop:
    del modified_manuf_dfs_new[key]

    
emptyDf=[key for key in modified_manuf_dfs_new.keys() if modified_manuf_dfs_new[key].shape[1]<4]

for dic in [modified_manuf_dfs_new]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''], errors = 'ignore')#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0
modified_manuf_dfs_new = dfSort(modified_manuf_dfs_new, client_manuf,f'{ManufOrTopC}',10, 'Total Value Share')
addOther(modified_manuf_dfs_new,col=f'{ManufOrTopC}',TorB="Bottom")


for s in manuf_P12M_dfs_new.keys():
    t = manuf_P12M_dfs_new[s].copy()
    t=DetectHeader(t).fillna(0)
    t.columns = t.columns.str.strip()            

    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        modified_manuf_P12M_new[s] = mod

df_toDrop =[key for key in modified_manuf_P12M_new.keys() if (modified_manuf_P12M_new[key].shape[1]>2) and (modified_manuf_P12M_new[key].shape[1]<4)]
for key in df_toDrop:
    del modified_manuf_P12M_new[key]


emptyDf=[key for key in modified_manuf_P12M_new.keys() if modified_manuf_P12M_new[key].shape[1]<4]
for dic in [modified_manuf_P12M_new]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0

modified_manuf_P12M_new = dfSort(modified_manuf_P12M_new, client_manuf,f'{ManufOrTopC}',10, 'Value Share')
addOther(modified_manuf_P12M_new,col=f"{ManufOrTopC}",TorB="Bottom")



{'Manual Shave Women | National': 3     Top Companies  Volume Sales P12M  Volume Sales LY  Value Sales P12M  \
 0  Procter & Gamble        189871045.0      183431260.0       569779501.0   
 7          Edgewell         90642693.0       89572460.0       272958900.0   
 1               Bic        175562218.0      176968282.0       148419913.0   
 2               Pbg         74183604.0       75417186.0        80784583.0   
 3            Harrys         23027376.0       20886525.0        68064716.0   
 4     Private Label         20253117.0       20166091.0        16640023.0   
 0            Others         30123031.0       28769948.0        59132391.0   
 
 3  Value Sales LY  Volume Share P12M  Volume Share LY  Value Share P12M  \
 0     551943535.0           0.314531         0.308178          0.468653   
 7     282253666.0           0.150154         0.150488          0.224513   
 1     155369970.0           0.290828         0.297320          0.122078   
 2      78707196.0           0.122889

In [11]:
modified_brands_share_new = {}
modified_brands_totals_new = {}
modified_brands_P12M_new = {}
modified_brands_evolution_new = {}
modified_manuf_evolution_new = {}


for s in brands_dfs_new.keys():
    t = brands_dfs_new[s].copy()
    t=DetectHeader(t).fillna(0)
    t.columns = t.columns.str.strip()            

    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        modified_brands_share_new[s] = mod
    if not tot.empty:
        modified_brands_totals_new[s] = tot


df_toDrop =[key for key in modified_brands_share_new.keys() if (modified_brands_share_new[key].shape[1]>2) and (modified_brands_share_new[key].shape[1]<4)]
for key in df_toDrop:
    del modified_brands_share_new[key]


emptyDf=[key for key in modified_brands_share_new.keys() if modified_brands_share_new[key].shape[1]<4]
for dic in [modified_brands_share_new]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0
p = 10 if len(client_brands) < 10 else len(client_brands) + 1

modified_brands_share_new = dfSort(modified_brands_share_new, client_brands,f'{BrandOrTopB}',p, 'Total Value Share')
for key in modified_brands_share_new.keys():
    modified_brands_share_new[key] = modified_brands_share_new[key][modified_brands_share_new[key]['Total Value Share'] != 0]
# Move rows containing "Others" in 'Top Brands' column to the first row
addOther(modified_brands_share_new,col=f'{BrandOrTopB}',TorB="Bottom")

for s in brands_P12M_dfs_new.keys():
    t = brands_P12M_dfs_new[s].copy()
    t=DetectHeader(t).fillna(0)
    t.columns = t.columns.str.strip()            
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    modified_brands_P12M_new[s] = mod

df_toDrop =[key for key in modified_brands_P12M_new.keys() if (modified_brands_P12M_new[key].shape[1]>2) and (modified_brands_P12M_new[key].shape[1]<4)]
for key in df_toDrop:
    del modified_brands_P12M_new[key]

emptyDf=[key for key in modified_brands_P12M_new.keys() if modified_brands_P12M_new[key].shape[1]<4]
for dic in [modified_brands_P12M_new]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0

modified_brands_P12M_new = dfSort(modified_brands_P12M_new, client_brands,f'{BrandOrTopB}',10, 'Value Share')
addOther(modified_brands_P12M_new,col=f'{BrandOrTopB}',TorB="Bottom")

    

{'Manual Shave Women | National': 2        Top Brands  Volume Sales  Value Sales  Volume Share  Value Share  \
 0    Gillette Venus   166766919.0  530970474.0      0.276258     0.436732   
 1        Bic Soleil    71855855.0  112654621.0      0.119033     0.092660   
 17        Intuition    22591056.0   88767757.0      0.037423     0.073013   
 16       Hydro Silk    24923524.0   84640725.0      0.041287     0.069618   
 2            Pbg Pl    74172400.0   80775340.0      0.122871     0.066439   
 3          Flamingo    23027376.0   68064716.0      0.038146     0.055984   
 9            Billie    23571159.0   66574255.0      0.039047     0.054758   
 4   Bic Silky Touch   101488355.0   30788631.0      0.168121     0.025324   
 31       Skintimate    14400013.0   22517211.0      0.023854     0.018521   
 0            Others    80866427.0  130026297.0      0.133960     0.106949   
 
 2   Share DYA  Av Price/KG  IYA Price/KG  
 0    0.012861     3.183908      0.996231  
 1   -0.007766     

In [12]:
def evolution_cleaning(dic,colname="Top Brands"):
    modified_evolution_new={}
    modified_evolution_sorted_new = {}
 
    for s in dic.keys():
        t = dic[s].copy()
        t=DetectHeader(t)
        t.columns = t.columns.str.strip()            
 
        t['Year'] = t['Year'].ffill()
        t = t.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
        t = t.applymap(lambda x: 0 if pd.isna(x) else x)
        mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
        mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
        tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
        if not mod.empty:
            modified_evolution_new[s] = mod
 
    for k in modified_evolution_new.keys():
        x = modified_evolution_new[k].copy()
        x[colname] = np.where(x[colname]==0, x['Year'], x[colname])
        x[colname] = x[colname].str.replace('2022 ','').str.replace('2021 ','').str.replace('2023 ',"").str.replace('2020 ','').str.replace('2024 ','').str.replace('2025 ','')
        x['Year'] = x['Year'].replace(0, np.nan).ffill().str.replace(' Total', '')
        x = x[x[colname] != 'Others']
        x = x[x[colname] != 'All Others']
        x = x[x['Price Evolution'] != 0]
 
        brand_sales = x.groupby(colname)['Value Sales'].sum().sort_values(ascending=False).head(6)
        if colname==f'BrandOrTopB':
            final = x[(x[colname].isin(pd.Series(brand_sales.index))) | (x[colname].isin(client_brands))]
        else:
            final = x[(x[colname].isin(pd.Series(brand_sales.index))) | (x[colname].isin(client_manuf))]
 
        final[['Price Evolution', 'Share Evolution']] = round(final[['Price Evolution', 'Share Evolution']].replace(np.nan,0).astype(float)*100).astype(int)
       
        final = final.drop_duplicates(subset = ["Year", colname])
        # final = final[final['Price Evolution'] != 0]
        # final = final[final['Top Brands'] != 'Others|All Others']
        final = final.sort_values('Year')
       
        modified_evolution_sorted_new[k] = final
    keys_to_remove = []
    for key, df in modified_evolution_sorted_new.items():
        if df.empty:  # Remove the () from df.empty
            keys_to_remove.append(key)
 
    for key in keys_to_remove:
        del modified_evolution_sorted_new[key]
 
    return modified_evolution_sorted_new

In [13]:
modified_brands_evolution_sorted_new=evolution_cleaning(brands_evolution_new,colname=f'{BrandOrTopB}')
modified_manuf_evolution_sorted_new=evolution_cleaning(manuf_evolution_new,colname=f'{ManufOrTopC}')

In [14]:
modified_retailer_regions = {}
modified_retailer_channels = {}
modified_retailer_markets = {}

modified_channels_regions = {}
modified_channels_channels = {}
modified_channels_markets = {}

modified_cust_regions = {}
modified_cust_channels = {}
modified_cust_markets = {}

dya_retailer={}
dya_channel={}
dya_cust = {}

#**********Retailer*********
if len(retailers)!=0:
    modified_retailer_regions, modified_retailer_channels, modified_retailer_markets, dya_retailer = process_RET_CHAN_POS(retailers_dfs_new, regions_RET, channels_RET, market_RET)
# *********Channels**********
if len(channels)!=0:
    modified_channels_regions, modified_channels_channels, modified_channels_markets, dya_channel = process_RET_CHAN_POS(channel_dfs_new, regions_CHAN, channels_CHAN, market_CHAN)
#********POS****************
if len(cust)!=0: 
    modified_cust_regions, modified_cust_channels, modified_cust_markets, dya_cust = process_RET_CHAN_POS(cust_dfs_new, regions_CUST, channels_CUST, market_CUST)


In [15]:
#### Client
modified_retailer_regions_client = {}
modified_retailer_channels_client = {}
modified_retailer_markets_client = {}

modified_channels_regions_client = {}
modified_channels_channels_client = {}
modified_channels_markets_client = {}

modified_cust_regions_client = {}
modified_cust_channels_client = {}
modified_cust_markets_client = {}


#**********Retailer*********
if len(retailers)!=0:
    modified_retailer_regions_client, modified_retailer_channels_client, modified_retailer_markets_client,_ = process_RET_CHAN_POS(retailers_client_dfs_new, regions_RET, channels_RET, market_RET)
# *********Channels**********
if len(channels)!=0:
    modified_channels_regions_client, modified_channels_channels_client, modified_channels_markets_client,_ = process_RET_CHAN_POS(channel_client_dfs_new, regions_CHAN, channels_CHAN, market_CHAN)
#********POS****************
if len(cust)!=0: 
    modified_cust_regions_client, modified_cust_channels_client, modified_cust_markets_client,_  = process_RET_CHAN_POS(cust_client_dfs_new, regions_CUST, channels_CUST, market_CUST)


In [16]:
modified_calendar_new = {}
for table in calendar_dfs_new.keys():
    modified_tables = []
    cleantable = calendar_dfs_new[table].copy()
    cleantable=DetectHeader(cleantable)
    cleantable =cleantable[cleantable['MonthYear']!='Grand Total']
    cleantable['time'] = pd.to_datetime(cleantable['MonthYear'], format='%b-%y')
    cleantable = cleantable.nlargest(25, 'time')
    cleantable = cleantable.sort_values('time')
    cleantable['Av Price/KG'] = cleantable['Av Price/KG'].replace(np.nan, 0)
    cleantable['Av Price/KG'] =round(cleantable['Av Price/KG'].astype(float),decimals)
    modified_calendar_new[table] = cleantable


keys_to_remove = []
for key, df in modified_calendar_new.items():
    if df.empty:  # Remove the () from df.empty
        keys_to_remove.append(key)

for key in keys_to_remove:
    del modified_calendar_new[key]


In [17]:
modified_categories_dfs_new=[]
for key in categories_overview_dfs_new.keys():
    try:
        dfCat=categories_overview_dfs_new[key].copy()
        dfCat=DetectHeader(dfCat)


        dfCat[['Cat Volume Sales/M','Cat Value Sales/M']]=round(dfCat[['Volume Sales','Value Sales']].astype(float)/10**6, 2)
        dfCat[['Cat Volume Sales Change YA','Cat Value Sales Change YA']]=round((dfCat[['Volume Sales IYA','Value Sales IYA']].astype(float)*100)-100,2)
        dfCat['Cat IYA Price/KG']=round((dfCat['IYA Price/KG'].astype(float)*100),decimals)
        dfCat['flag']=1

        dfMamu=categories_overview_manuf_dfs_new[key].copy()
        dfMamu=DetectHeader(dfMamu)

        
        dfMamu[['Manu Volume Sales/M','Manu Value Sales/M']]=round(dfMamu[['Volume Sales','Value Sales']].astype(float)/10**6, 2)
        dfMamu[['Manu Volume Sales Change YA','Manu Value Sales Change YA']]=round((dfMamu[['Volume Sales IYA','Value Sales IYA']].astype(float)*100)-100,2)
        dfMamu['Manu IYA Price/KG']=round((dfMamu['IYA Price/KG'].astype(float)*100),2)
        dfMamu['Impact Growth'] = dfMamu['Manu Value Sales Change YA'] - dfMamu['Manu Volume Sales Change YA']
        dfMamu['flag']=1

        table = pd.merge(dfCat[['flag','Cat Volume Sales/M','Cat Value Sales/M','Cat Volume Sales Change YA','Cat Value Sales Change YA','Cat IYA Price/KG']],dfMamu[['flag','Manu Volume Sales/M','Manu Value Sales/M','Manu Volume Sales Change YA','Manu Value Sales Change YA','Manu IYA Price/KG','Impact Growth']]).drop(columns='flag')
        table = table[['Cat Value Sales/M','Cat Value Sales Change YA','Cat IYA Price/KG', 'Manu Value Sales/M','Manu Value Sales Change YA','Manu IYA Price/KG','Cat Volume Sales/M','Cat Volume Sales Change YA','Manu Volume Sales/M','Manu Volume Sales Change YA','Impact Growth']]
        table['Price Impact'] = str(int(round(table['Manu Value Sales Change YA']))-int(round(table['Manu Volume Sales Change YA'])))+"pts"
        
        table['Cat Value Sales Change YA']=str(int(round(table['Cat Value Sales Change YA'])))+'%'
        table['Cat Volume Sales Change YA']=str(int(round(table['Cat Volume Sales Change YA'])))+'%'
        table['Manu Value Sales Change YA']=str(int(round(table['Manu Value Sales Change YA'])))+'%'
        table['Manu Volume Sales Change YA']=str(int(round(table['Manu Volume Sales Change YA'])))+'%'
        table['Cat IYA Price/KG']=str(int(round(table['Cat IYA Price/KG'])))+'%'
        table['Manu IYA Price/KG']=str(int(round(table['Manu IYA Price/KG'])))+'%'
        lis=['Cat Value Sales/M','Manu Value Sales/M','Cat Volume Sales/M','Manu Volume Sales/M','Impact Growth']
        for col in lis:
            if round(table[col][0],1)==int(round(table[col][0],1)):
                table[col]=int(round(table[col]))
            else:
                table[col]=round(table[col],1)
        
        dfVal=categories_values_dfs_new[key].copy()
        dfVal=DetectHeader(dfVal)

        dfVal['MonthYear']=dfVal['MonthYear'].ffill()

        modified_categories_dfs_new.append([table,dfVal,key])
    # except:
        # print(key)
    except Exception as e:
        print(f"Error with key '{key}': {e}")
    

In [18]:
sharGrowthDf_sec={}
sharGrowthDf_seg={}
sharGrowthDf_subseg={}
sharGrowthDf_subcat={}
shareGrowthDf_retailers_REG={}
shareGrowthDf_retailers_CHAN={}
shareGrowthDf_retailers_MAR={}
shareGrowthDf_channels_REG={}
shareGrowthDf_channels_CHAN={}
shareGrowthDf_channels_MAR={}
shareGrowthDf_custom_REG={}
shareGrowthDf_custom_CHAN={}
shareGrowthDf_custom_MAR={}

for key in categories + sectors + segments + subsegments +subcategories:
    
    if key in categories:
        shareGrowthDf_retailers_REG[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in regions_RET ] 
        shareGrowthDf_retailers_CHAN[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in channels_RET ]
        shareGrowthDf_retailers_MAR[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in market_RET ] 
        
        shareGrowthDf_channels_REG[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in regions_CHAN ] 
        shareGrowthDf_channels_CHAN[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in channels_CHAN ]
        shareGrowthDf_channels_MAR[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in market_CHAN ]

        shareGrowthDf_custom_REG[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in regions_CUST ] 
        shareGrowthDf_custom_CHAN[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in channels_CUST ]
        shareGrowthDf_custom_MAR[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in market_CUST ]

    else:
        shareGrowthDf_retailers_REG[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in regions_RET ] 
        shareGrowthDf_retailers_CHAN[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in channels_RET ]
        shareGrowthDf_retailers_MAR[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in market_RET ] 

        shareGrowthDf_channels_REG[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in regions_CHAN ]
        shareGrowthDf_channels_CHAN[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in channels_CHAN ]
        shareGrowthDf_channels_MAR[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in market_CHAN ]

        shareGrowthDf_custom_REG[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in regions_CUST ] 
        shareGrowthDf_custom_CHAN[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in channels_CUST ]
        shareGrowthDf_custom_MAR[key]=[k.split(' | ')[0] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[1] and k.split(' | ')[0] in market_CUST ]

    if len(shareGrowthDf_retailers_REG[key])==0:
        shareGrowthDf_retailers_REG.pop(key,None)    
    if len(shareGrowthDf_retailers_CHAN[key])==0:
        shareGrowthDf_retailers_CHAN.pop(key,None)
    if len(shareGrowthDf_retailers_MAR[key])==0:
        shareGrowthDf_retailers_MAR.pop(key,None)
    
    if len(shareGrowthDf_channels_REG[key])==0:
        shareGrowthDf_channels_REG.pop(key,None)    
    if len(shareGrowthDf_channels_CHAN[key])==0:
        shareGrowthDf_channels_CHAN.pop(key,None)
    if len(shareGrowthDf_channels_MAR[key])==0:
        shareGrowthDf_channels_MAR.pop(key,None)

    if len(shareGrowthDf_custom_REG[key])==0:
        shareGrowthDf_custom_REG.pop(key,None)    
    if len(shareGrowthDf_custom_CHAN[key])==0:
        shareGrowthDf_custom_CHAN.pop(key,None)
    if len(shareGrowthDf_custom_MAR[key])==0:
        shareGrowthDf_custom_MAR.pop(key,None)
    

for key in ["National"] + regions_RET + channels_RET+market_RET +  regions_CHAN +channels_CHAN+ market_CHAN+regions_CUST+channels_CUST+ market_CUST:
    sharGrowthDf_sec[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in sectors ]
    sharGrowthDf_seg[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in segments]
    sharGrowthDf_subseg[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in subsegments]
    sharGrowthDf_subcat[key]=[k.split(' | ')[1] for k in modified_manuf_P12M_new.keys() if key == k.split(' | ')[0] and k.split(' | ')[1] in subcategories]
    
    if len(sharGrowthDf_sec[key])==0:
        sharGrowthDf_sec.pop(key,None)
        
    if len(sharGrowthDf_seg[key])==0:
        sharGrowthDf_seg.pop(key,None)
    
    if len(sharGrowthDf_subseg[key])==0:
        sharGrowthDf_subseg.pop(key,None)
    
    if len(sharGrowthDf_subcat[key])==0:
        sharGrowthDf_subcat.pop(key,None)

### Create Market Growth Market List for duplication and replacement

In [19]:
markets_name = ['regions_RET' , 'channels_RET','market_RET'  ,'regions_CHAN' ,'channels_CHAN', 'market_CHAN' ,'regions_CUST' ,'channels_CUST', 'market_CUST']
markets_val = [regions_RET , channels_RET,market_RET  ,regions_CHAN ,channels_CHAN, market_CHAN ,regions_CUST ,channels_CUST, market_CUST]
markets_dfs = [modified_retailer_regions , modified_retailer_channels,modified_retailer_markets  ,modified_channels_regions ,modified_channels_channels,  modified_channels_markets,modified_cust_regions ,modified_cust_channels, modified_cust_markets]
market_total = [dya_retailer,dya_channel,dya_cust]
slide_by = ['Region','Channel','Market']*3
slide_for = [*['Retailer']*3,*['Channel']*3,*[customareas]*3]
market_list_Growth={}
duplication_num = []
section_name_Growth = []
for idx,val in enumerate(markets_val):
    if len(val)>0:
        market_list_Growth[markets_name[idx]]=[val,markets_dfs[idx],market_total[idx//3],slide_by[idx],slide_for[idx]]
        duplication_num.append(len(markets_dfs[idx].keys()))
        section_name_Growth.append('Market Growth By '+slide_for[idx]+' For '+slide_by[idx])
section_number = len(market_list_Growth.keys())


#####  Create Value_AvgPrice Market List for duplication and replacement

In [20]:
markets_name = ['regions_RET' , 'channels_RET','market_RET'  ,'regions_CHAN' ,'channels_CHAN', 'market_CHAN' ,'regions_CUST' ,'channels_CUST', 'market_CUST']
markets_val = [regions_RET , channels_RET,market_RET  ,regions_CHAN ,channels_CHAN, market_CHAN ,regions_CUST ,channels_CUST, market_CUST]
markets_dfs = [modified_retailer_regions , modified_retailer_channels,modified_retailer_markets  ,modified_channels_regions ,modified_channels_channels,  modified_channels_markets,modified_cust_regions ,modified_cust_channels, modified_cust_markets]
client_dfs = [modified_retailer_regions_client , modified_retailer_channels_client,modified_retailer_markets_client  ,modified_channels_regions_client ,modified_channels_channels_client,  modified_channels_markets_client,modified_cust_regions_client ,modified_cust_channels_client, modified_cust_markets_client]
market_total = [dya_retailer,dya_channel,dya_cust]
slide_by = ['Region','Channel','Market']*3
slide_for = [*['Retailer']*3,*['Channel']*3,*[customareas]*3]
market_list_Avg={}
duplication_num_Avg = []
section_name_Avg = []


for idx,val in enumerate(markets_val):
    if len(val)>0:
        market_list_Avg[markets_name[idx]]=[val,markets_dfs[idx],client_dfs[idx],slide_by[idx],slide_for[idx]]
        duplication_num_Avg.append(len(client_dfs[idx].keys()))
        section_name_Avg.append('Value Vs AvgPrice By '+slide_for[idx]+' For '+slide_by[idx])
section_number_Avg = len(market_list_Avg.keys())

### Slide 12 : Category Evolution Analysis 

In [21]:
def cleanSectorSegment(dfDic):
    catDic, totalDic = {}, {}
    for key, df in dfDic.items():
        df = df.copy()
        catIndex = df.index[df['Unnamed: 0'].isin(['Sector', 'Segment', 'SubSegment', 'SubCategory'])][0]
        catName = df['Unnamed: 0'][catIndex]
        df.columns = (df.iloc[catIndex].replace(np.nan, '') + ' ' + df.iloc[catIndex-1].ffill().replace(np.nan, "")).str.replace(' Values', '').str.strip()
        df = df.iloc[catIndex + 1:]
        df[catName] = df[catName].ffill()

        df = df.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
        df = df.applymap(lambda x: 0 if pd.isna(x) else x)

        df[catName] = df[catName].astype(str)
        addative = df[~df[catName].str.contains(' Total')].sort_values('WoB %', ascending=False)

        total = df[df[catName].str.contains(' Total')]
        total[catName] = total[catName].str.replace(' Total', '')

        # Check if the total DataFrame is empty and if so, add a "Grand" total row with zeros
        if total.empty:
            zero_row = pd.Series([0] * (len(df.columns) - 1), index=df.columns[1:])
            zero_row[catName] = "Grand"
            total = pd.concat([total, zero_row.to_frame().T], ignore_index=True)
        if not addative.empty:
            catDic[key.replace('  ', ' ')] = addative
        if not total.empty:
            totalDic[key.replace('  ', ' ')] = total

    return catDic, totalDic


In [22]:
cea_sector_P12M_new,cea_dya_sector_new,cea_sector_clients_new,cea_sector_clients_total_new ={},{},{},{}

if sectors:
    cea_sectors_P12M_new,cea_dya_sectors_new=cleanSectorSegment(sectors_P12M_dfs)
    cea_sectors_clients_new,cea_sectors_clients_total_new=cleanSectorSegment(sectors_client_dfs_category)

cea_segment_P12M_new,cea_dya_segment_new,cea_segment_clients_new,cea_segment_clients_total_new ={},{},{},{}

if segments:
    cea_segment_P12M_new,cea_dya_segment_new=cleanSectorSegment(segments_P12M_dfs)
    cea_segment_clients_new,cea_segment_clients_total_new=cleanSectorSegment(segments_client_dfs_category)

In [23]:
cea_subsegment_P12M_new,cea_dya_subsegment_new,cea_subsegment_clients_new,cea_subsegment_clients_total_new ={},{},{},{}
if subsegments:
    cea_subsegment_P12M_new,cea_dya_subsegment_new=cleanSectorSegment(subsegments_P12M_dfs_new)
    cea_subsegment_clients_new,cea_subsegment_clients_total_new=cleanSectorSegment(subsegments_client_dfs_category)

cea_subcategory_P12M_new,cea_dya_subcategory_new,cea_subcategory_clients_new,cea_subcategory_clients_total_new ={},{},{},{}
if subcategories:
    cea_subcategory_P12M_new,cea_dya_subcategory_new=cleanSectorSegment(subcategories_P12M_dfs_new)
    cea_subcategory_clients_new,cea_subcategory_clients_total_new=cleanSectorSegment(subcategories_client_dfs_category)

## Slide 13 : Share And Growth

In [24]:
retailerRegion,retailerChannel,retailertMarket,retailerTotal=process_RET_CHAN_POS(retailers_dfs_new,regions_RET,channels_RET,market_RET)
channelRegion,channelChannel,channelMarket,channelTotal=process_RET_CHAN_POS(channel_dfs_new,regions_CHAN,channels_CHAN,market_CHAN)
custRegion,custChannel,custMarket,custTotal=process_RET_CHAN_POS(cust_dfs_new,regions_CUST,channels_CUST,market_CUST)

In [25]:
clientRetailerRegion,clientretailerChannel,clientRetailertMarket,retailerClientTotal=process_RET_CHAN_POS(retailers_client_dfs_new,regions_RET,channels_RET,market_RET)
clientChannelRegion,clientChannelChannel,clientChannelMarket,channelClientTotal=process_RET_CHAN_POS(channel_client_dfs_new,regions_CHAN,channels_CHAN,market_CHAN)
clientCustRegion,clientCustChannel,clientCustMarket,custClientTotal=process_RET_CHAN_POS(cust_client_dfs_new,regions_CUST,channels_CUST,market_CUST)

In [26]:
retailerCatDic,retailerSecDic,retailerSegDic,retailerSubSegDic,retailerSubCatDic=[],[],[],[],[]
channelCatDic,channelSecDic,channelSegDic,channelSubSegDic,channelSubCatDic=[],[],[],[],[]
custCatDic,custSecDic,custSegDic,custSubSegDic,custSubCatDic=[],[],[],[],[]

if retailers_dfs_new:

    retailerCatDic = [key for key in retailerClientTotal.keys() if (len(set(categories).intersection(set(key.split(' | '))))) ]
    retailerSecDic = [key for key in retailerClientTotal.keys() if len(set(sectors).intersection(set(key.split(' | '))))]
    retailerSegDic = [key for key in retailerClientTotal.keys() if len(set(segments).intersection(set(key.split(' | '))))]
    retailerSubSegDic = [key for key in retailerClientTotal.keys() if len(set(subsegments).intersection(set(key.split(' | '))))]
    retailerSubCatDic = [key for key in retailerClientTotal.keys() if len(set(subcategories).intersection(set(key.split(' | '))))]



if channel_dfs_new:

    channelCatDic = [key for key in channelClientTotal.keys() if len(set(categories).intersection(set(key.split(' | '))))]
    channelSecDic = [key for key in channelClientTotal.keys() if len(set(sectors).intersection(set(key.split(' | '))))]
    channelSegDic = [key for key in channelClientTotal.keys() if len(set(segments).intersection(set(key.split(' | '))))]
    channelSubSegDic = [key for key in channelClientTotal.keys() if len(set(subsegments).intersection(set(key.split(' | '))))]
    channelSubCatDic = [key for key in channelClientTotal.keys() if len(set(subcategories).intersection(set(key.split(' | '))))]

if cust_dfs_new:

    custCatDic = [key for key in custClientTotal.keys() if len(set(categories).intersection(set(key.split(' | ')))) ]
    custSecDic = [key for key in custClientTotal.keys() if len(set(sectors).intersection(set(key.split(' | ')))) ]
    custSegDic = [key for key in custClientTotal.keys() if len(set(segments).intersection(set(key.split(' | '))))]
    custSubSegDic = [key for key in custClientTotal.keys() if len(set(subsegments).intersection(set(key.split(' | '))))]
    custSubCatDic = [key for key in custClientTotal.keys() if len(set(subcategories).intersection(set(key.split(' | '))))]



In [27]:
retailerField = {'Region':[retailerRegion,clientRetailerRegion,retailerTotal,retailerClientTotal],'Channel':[retailerChannel,clientretailerChannel,retailerTotal,retailerClientTotal],'Market':[retailertMarket,clientRetailertMarket,retailerTotal,retailerClientTotal]}
channelField = {'Region':[channelRegion,clientChannelRegion,channelTotal,channelClientTotal],'Channel':[channelChannel,clientChannelChannel,channelTotal,channelClientTotal],'Market':[channelMarket,clientChannelMarket,channelTotal,channelClientTotal]}
custField = {'Region':[custRegion,clientCustRegion,custTotal,custClientTotal],'Channel':[custChannel,clientCustChannel,custTotal,custClientTotal],'Market':[custMarket,clientCustMarket,custTotal,custClientTotal]}

### Slide 14: Momentum Analysis

In [28]:
modified_momentum_retailers = process_RET_CHAN_Momentum(momentum_retailers_dfs, regions_RET,channels_RET,market_RET,client_brands, "Retailers")

modified_momentum_channels = process_RET_CHAN_Momentum(momentum_channel_dfs, regions_CHAN,channels_CHAN,market_CHAN,client_brands, "Channels")

modified_momentum_cust = process_RET_CHAN_Momentum(momentum_cust_dfs, regions_CUST,channels_CUST,market_CUST,client_brands, f"{customareas}")


In [29]:
if segments:
    if retailers:
        modified_momentum_retailers_seg = process_RET_CHAN_Momentum_Sector(momentum_retailers_seg_dfs, regions_RET,channels_RET,market_RET,client_brands, "Retailers","Segment", segments)
    if channels:
        modified_momentum_channels_seg = process_RET_CHAN_Momentum_Sector(momentum_channels_seg_dfs,regions_CHAN,channels_CHAN,market_CHAN,client_brands, "Channels","Segment", segments)
    if cust:
        modified_momentum_cust_seg = process_RET_CHAN_Momentum_Sector(momentum_cust_seg_dfs, regions_CUST,channels_CUST,market_CUST,client_brands, f"{customareas}","Segment", segments)


if subsegments:
    if retailers:
        modified_momentum_retailers_subseg = process_RET_CHAN_Momentum_Sector(momentum_retailers_subseg_dfs, regions_RET,channels_RET,market_RET,client_brands, "Retailers","SubSegment", subsegments)
    if channels:
        modified_momentum_channels_subseg = process_RET_CHAN_Momentum_Sector(momentum_channels_subseg_dfs,regions_CHAN,channels_CHAN,market_CHAN,client_brands, "Channels","SubSegment", subsegments)
    if cust:
        modified_momentum_cust_subseg = process_RET_CHAN_Momentum_Sector(momentum_cust_subseg_dfs, regions_CUST,channels_CUST,market_CUST,client_brands, f"{customareas}","SubSegment", subsegments)


if subcategories:
    if retailers:
        modified_momentum_retailers_subcatg = process_RET_CHAN_Momentum_Sector(momentum_retailers_subcatg_dfs, regions_RET,channels_RET,market_RET,client_brands, "Retailers","SubCategory", subcategories)
    if channels:
        modified_momentum_channels_subcatg = process_RET_CHAN_Momentum_Sector(momentum_channels_subcatg_dfs,regions_CHAN,channels_CHAN,market_CHAN,client_brands, "Channels","SubCategory", subcategories)
    if cust:
        modified_momentum_cust_subcatg = process_RET_CHAN_Momentum_Sector(momentum_cust_subcatg_dfs, regions_CUST,channels_CUST,market_CUST,client_brands, f"{customareas}","SubCategory", subcategories)


In [30]:
modified_momentum_dfs = {}
for key in momentum_dfs.keys():
    t = momentum_dfs[key].copy()
    # t=DetectHeader(t)
    t.iloc[15] = t.iloc[15].ffill().replace(np.nan, "") 
    t.columns = (t.iloc[16] +" "+ t.iloc[15]).str.strip()
    t = t.iloc[17:]
    if not t.empty:
        t[f'{BrandOrTopB}'] = t[f'{BrandOrTopB}'].ffill()
        if not t.empty:
            top_brands_totals = t[(t[f'{BrandOrTopB}'].str.contains("Total")) & (t[f'{BrandOrTopB}']!="Grand Total|All Others Total") & (~t[f"{BrandOrTopB}"].isin([i+" Total" for i in client_brands]))]
            competitors = top_brands_totals.sort_values(by = "Value Sales P12M",ascending=False).head(5)[f'{BrandOrTopB}'].to_list()
            competitors = [i.replace(" Total","") for i in competitors]

            t_filtered = t[t[f'{BrandOrTopB}'].isin(client_brands) |
                                    ((t[f'{BrandOrTopB}'].isin(competitors)) & (t['Value Share P12M'] > 0.02))]

            t_filtered = t_filtered.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
            t_filtered['Value Sales P12M'] = t_filtered['Value Sales P12M'].astype(float)
            t_filtered = t_filtered.sort_values('Value Sales P12M', ascending=False)
            t_filtered['Labels'] = t_filtered[f'{BrandOrTopB}'] + "/" + t_filtered['Sector'] 

            if not t_filtered.empty:
                modified_momentum_dfs[key] = t_filtered 

In [31]:
def genrateIndexList(scopeCategory ,chartIndex =0 ,chartCount =4 ):
    if len(scopeCategory) == 0:
        return []
    page = len(scopeCategory) # number of sector or segment or category
 
    lis=[]
    while page >chartCount:
        lis.append(chartCount + chartIndex)
        page -=chartCount
    lis.append(page + chartIndex)
    return [lis]

sectorIndex = []
segmentIndex = []
subsegmentIndex = []
subcategoryIndex = []
retailerREGIndex = []
retailerCHAIndex = []
retailerMARIndex = []
channelREGIndex = []
channelCHAIndex = []
channelMARIndex = []
custREGIndex = []
custCHAIndex = []
custMARIndex = []

if len (sectors)!=0:
    
    for key,value in sharGrowthDf_sec.items():
        sectorIndex+=genrateIndexList(value,4,3)[0]

if len(segments)!=0:
    
    for key,value in sharGrowthDf_seg.items():
        segmentIndex+=genrateIndexList(value,4,3)[0]

if len(subsegments)!=0:
    
    for key,value in sharGrowthDf_subseg.items():
        subsegmentIndex+=genrateIndexList(value,4,3)[0]

if len(subcategories)!=0:
    
    for key,value in sharGrowthDf_subcat.items():
        subcategoryIndex+=genrateIndexList(value,4,3)[0]

if len(regions_RET)!=0:
    for key,value in shareGrowthDf_retailers_REG.items():
        retailerREGIndex+=genrateIndexList(value,4,3)[0]

if len(channels_RET)!=0:
    for key,value in shareGrowthDf_retailers_CHAN.items():
        retailerCHAIndex+=genrateIndexList(value,4,3)[0]
if len(market_RET)!=0:
    for key,value in shareGrowthDf_retailers_MAR.items():
        retailerMARIndex+=genrateIndexList(value,4,3)[0]
        
if len(regions_CHAN)!=0:
    for key,value in shareGrowthDf_channels_REG.items():
        channelREGIndex+=genrateIndexList(value,4,3)[0]
if len(channels_CHAN)!=0:
    for key,value in shareGrowthDf_channels_CHAN.items():
        channelCHAIndex+=genrateIndexList(value,4,3)[0]
if len(market_CHAN)!=0:
    for key,value in shareGrowthDf_channels_MAR.items():
        channelMARIndex+=genrateIndexList(value,4,3)[0]

if len(regions_CUST)!=0:
    for key,value in shareGrowthDf_custom_REG.items():
        custREGIndex+=genrateIndexList(value,4,3)[0]
if len(channels_CUST)!=0:
    for key,value in shareGrowthDf_custom_CHAN.items():
        custCHAIndex+=genrateIndexList(value,4,3)[0]
if len(market_CUST)!=0:
    for key,value in shareGrowthDf_custom_MAR.items():
        custMARIndex+=genrateIndexList(value,4,3)[0]


# Slide Creations

### First Landscape Deck

In [94]:
index = [
    *[0]*(2+(1 if sectors else 0)+(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
           *[1]*(2+(1 if sectors else 0)+(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
               *[2]*((1 if sectors else 0) +(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
               *[2]*section_number,
               *[3]*((1 if sectors else 0) +(1 if segments else 0)+(1 if subcategories else 0)+(1 if subsegments else 0)),
             *[3]*section_number_Avg,
                ]
index=[i for i in index if i !=[]]

duplication_1 = [len(modified_manuf_dfs_new.keys()), len(modified_brands_share_new.keys()),len(modified_sectors_dfs_new.keys())if sectors else 0,len(modified_segment_dfs_new.keys())if segments else 0,len(modified_subsegment_dfs_new.keys())if subsegments else 0,len(modified_subcategories_dfs_new.keys())if subcategories else 0]
duplication_2 = [len(modified_manuf_dfs_new.keys()), len(modified_brands_share_new.keys()), len(modified_sectors_dfs_new.keys())if sectors else 0, len(modified_segment_dfs_new.keys())if segments else 0,len(modified_subsegment_dfs_new.keys())if subsegments else 0,len(modified_subcategories_dfs_new.keys())if subcategories else 0]
duplication_3 = [len(modified_sectors_P12M_new.keys()) if sectors else 0, len(modified_segment_P12M_new.keys())if segments else 0, len(modified_subsegment_P12M_new.keys())if subsegments else 0, len(modified_subcategories_P12M_new.keys())if subcategories else 0,*duplication_num]
duplication_4 = [len(modified_sectors_clients_new.keys())if sectors else 0,len(modified_segment_clients_new.keys())if segments else 0, len(modified_subsegment_clients_new.keys())if subsegments else 0, len(modified_subcategories_clients_new.keys())if subcategories else 0, *duplication_num_Avg]

duplication = duplication_1 + duplication_2  + duplication_3  + duplication_4
duplication = [item for item in duplication if item !=0]

section_names_slide1 = ["Market Trends by Manufacturer","Market Trends by Brands","Market Trends by Sectors"] + (["Market Trends by Segments"] if len(segments)>0 else [])+(["Market Trends by SubSegments"] if len(subsegments)>0 else [])+(["Market Trends by SubCategory"] if len(subcategories)>0 else [])
section_names_slide2 = ["Market Concentration By Manufacturer", "Market Concentration By Brands", "Market Concentration By Sectors"]+ (["Market Concentration By Segments"] if len(segments)>0 else [])+(["Market Concentration by SubSegments"] if len(subsegments)>0 else [])+(["Market Concentration by SubCategory"] if len(subcategories)>0 else [])
section_names_slide3 = (["Market Growth By Sectors"]if len(sectors)>0 else [])+(["Market Growth By Segments"]if len(segments)>0 else [])+(["Market Growth By SubSegments"]if len(subsegments)>0 else [])+(["Market Growth By SubCategory"]if len(subcategories)>0 else [])+[*section_name_Growth]
section_names_slide4 = (["Value Vs AvgPrice By Sectors"]if len(sectors)>0 else [])+(["Value Vs AvgPrice By Segments"]if len(segments)>0 else [])+(["Value Vs AvgPrice By SubSegments"]if len(subsegments)>0 else [])+(["Value Vs AvgPrice By SubCategory"]if len(subcategories)>0 else [])+[*section_name_Avg]


section_names = [ *section_names_slide1 ,*section_names_slide2,*section_names_slide3, *section_names_slide4                   
                 ]
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape duplicate Market(Trends).pptx'



In [95]:
print(len(index))
print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
# print(section_names)

24
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3]
24
[175, 175, 10, 10, 10, 10, 175, 175, 10, 10, 10, 10, 10, 10, 10, 10, 18, 18, 50, 50, 50, 50, 66, 66]
24


In [96]:
print(sum(duplication))

1188


In [34]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [97]:
prs = Presentation(new_pre)

In [98]:
def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

#### Slide 1_ Market_Trends

In [99]:
p=0
Market_Trends(prs, list(modified_manuf_dfs_new.keys()), modified_manuf_dfs_new, modified_manuf_totals_new, client_manuf ,position = calculate_position(p), slide_by = f'{ManufOrTopC}', years=years)
p+=1

Market_Trends(prs, list(modified_brands_share_new.keys()), modified_brands_share_new, modified_brands_totals_new, client_brands ,position =calculate_position(p), slide_by = f'{BrandOrTopB}', years=years)
p+=1

if len(sectors)!=0:
    Market_Trends(prs, list(modified_sectors_dfs_new.keys()), modified_sectors_dfs_new, sectors_totals_new, sectors ,position = calculate_position(p), slide_by = 'Sector', years=years)
    p+=1

if len(segments)!=0:
    Market_Trends(prs, list(modified_segment_dfs_new.keys()), modified_segment_dfs_new, segment_totals_new, segments ,position = calculate_position(p), slide_by = 'Segment', years=years)
    p+=1

    
if len(subsegments)!=0:
    Market_Trends(prs, list(modified_subsegment_dfs_new.keys()), modified_subsegment_dfs_new, subsegment_totals_new, subsegments ,position = calculate_position(p), slide_by = 'SubSegment', years=years)
    p+=1

if len(subcategories)!=0:
    Market_Trends(prs, list(modified_subcategories_dfs_new.keys()), modified_subcategories_dfs_new, subcategories_totals_new, subcategories ,position = calculate_position(p), slide_by = 'SubCategory', years=years)
    p+=1


In [100]:
for k in modified_brands_share_new.keys():
    modified_brands_share_new[k][f'{BrandOrTopB}'] = modified_brands_share_new[k][f'{BrandOrTopB}'].str.replace("Private Label", "PL")

#### Slide 2_Market_Concentration

In [101]:
Market_Concentration(prs,list(modified_manuf_dfs_new.keys()), modified_manuf_dfs_new, modified_manuf_totals_new, client_manuf ,position =  calculate_position(p), slide_by = f'{ManufOrTopC}',years=years)
p+=1
Market_Concentration(prs,list(modified_brands_share_new.keys()), modified_brands_share_new, modified_brands_totals_new, client_brands ,position =  calculate_position(p), slide_by = f'{BrandOrTopB}',years=years)
p+=1
if len(sectors)!=0:
    Market_Concentration(prs,list(modified_sectors_dfs_new.keys()), modified_sectors_dfs_new, sectors_totals_new, sectors ,position =  calculate_position(p), slide_by = 'Sector',years=years)
    p+=1
if len(segments)!=0:
    Market_Concentration(prs,list(modified_segment_dfs_new.keys()), modified_segment_dfs_new, segment_totals_new, segments ,position =  calculate_position(p), slide_by = 'Segment',years=years)
    p+=1

if len(subsegments)!=0:
    Market_Concentration(prs,list(modified_subsegment_dfs_new.keys()), modified_subsegment_dfs_new, subsegment_totals_new, subsegments ,position =  calculate_position(p), slide_by = 'SubSegment',years=years)
    p+=1

if len(subcategories)!=0:
    Market_Concentration(prs,list(modified_subcategories_dfs_new.keys()), modified_subcategories_dfs_new, subcategories_totals_new, subcategories ,position =  calculate_position(p), slide_by = 'SubCategory',years=years)
    p+=1


#### Slide 3_Market_Growth_Contribution

###### By Categpry/Sec/Seg/SubSeg/SubCat

In [102]:
Market_Growth(prs,list(modified_sectors_P12M_new.keys()), modified_sectors_P12M_new, dya_sectors_new, position = calculate_position(p) , slide_by = 'Sector')
p+=1
if len(segments) !=0:
    Market_Growth(prs,list(modified_segment_P12M_new.keys()), modified_segment_P12M_new, dya_segment_new, position = calculate_position(p) , slide_by = 'Segment') 
    p+=1

if len(subsegments) !=0:
    Market_Growth(prs,list(modified_subsegment_P12M_new.keys()), modified_subsegment_P12M_new, dya_subsegment_new, position = calculate_position(p) , slide_by = 'SubSegment') 
    p+=1

if len(subcategories) !=0:
    Market_Growth(prs,list(modified_subcategories_P12M_new.keys()), modified_subcategories_P12M_new, dya_subcategories_new, position = calculate_position(p) , slide_by = 'SubCategory') 
    p+=1


###### By Region/Channel/Market

In [103]:
for key,val in market_list_Growth.items():
    Market_Growth(prs, list(val[1].keys()), val[1], val[2], position = calculate_position(p), slide_by = val[3], slide_for = val[4])
    p+=1


#### Slide4_ValueSales_AvgPrice

In [108]:
market_list=["National"] + regions_RET + channels_RET+market_RET +  regions_CHAN +channels_CHAN+ market_CHAN+regions_CUST +channels_CUST+ market_CUST
manuf_brand_list = client_manuf + client_brands

if len(sectors):
    ValueSales_AvgPrice(prs, list(modified_sectors_clients_new.keys()), modified_sectors_P12M_new, modified_sectors_clients_new,manuf_brand_list, market_list,position = calculate_position(p), slide_by = 'Sector')
    ValueSales_AvgPrice(prs, list(modified_sectors_clients_new.keys()), modified_sectors_P12M_new, modified_sectors_clients_new,manuf_brand_list, market_list,position = calculate_position(p), slide_by = 'Sector')
    p+=1

if len(segments)!=0:
    ValueSales_AvgPrice(prs, list(modified_segment_clients_new.keys()), modified_segment_P12M_new, modified_segment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'Segment')
    ValueSales_AvgPrice(prs, list(modified_segment_clients_new.keys()), modified_segment_P12M_new, modified_segment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'Segment')
    p+=1

if len(subsegments)!=0:
    ValueSales_AvgPrice(prs, list(modified_subsegment_clients_new.keys()), modified_subsegment_P12M_new, modified_subsegment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubSegment')
    ValueSales_AvgPrice(prs, list(modified_subsegment_clients_new.keys()), modified_subsegment_P12M_new, modified_subsegment_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubSegment')
    p+=1

if len(subcategories)!=0:
    ValueSales_AvgPrice(prs, list(modified_subcategories_clients_new.keys()), modified_subcategories_P12M_new, modified_subcategories_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubCategory')
    ValueSales_AvgPrice(prs, list(modified_subcategories_clients_new.keys()), modified_subcategories_P12M_new, modified_subcategories_clients_new,manuf_brand_list,market_list, position = calculate_position(p) , slide_by = 'SubCategory')
    p+=1

catg_list=[f'{categories[0]}'] + sectors + segments + subsegments + subcategories
for key,val in market_list_Avg.items():
    ValueSales_AvgPrice(prs, list(val[2].keys()), val[1], val[2],manuf_brand_list,catg_list ,position = calculate_position(p), slide_by = val[3], slide_for = val[4])
    ValueSales_AvgPrice(prs, list(val[2].keys()), val[1], val[2],manuf_brand_list,catg_list ,position = calculate_position(p), slide_by = val[3], slide_for = val[4])
    p+=1

In [109]:
for k in modified_brands_P12M_new.keys():
    modified_brands_P12M_new[k] = modified_brands_P12M_new[k].rename(columns = { "Value Share P12M":"Value Share", "Share DYA P12M":"Share DYA", "Av Price/KG P12M":"Av Price/KG"})
    modified_brands_P12M_new[k] = modified_brands_P12M_new[k].sort_values('Value Share', ascending = False)
    modified_brands_P12M_new[k] = modified_brands_P12M_new[k][modified_brands_P12M_new[k]['Value Share']!=0]    
    modified_brands_P12M_new[k]['Sorting'] = np.where(modified_brands_P12M_new[k][f'{BrandOrTopB}'] == 'Others', 1, 0)
    modified_brands_P12M_new[k] = modified_brands_P12M_new[k].sort_values(['Sorting','Value Share'], ascending = [True, False]).drop(columns = 'Sorting')
    
for k in modified_manuf_P12M_new.keys():
    modified_manuf_P12M_new[k] = modified_manuf_P12M_new[k].sort_values('Total Value Share', ascending=False)
    modified_manuf_P12M_new[k]['Sorting'] = np.where(modified_manuf_P12M_new[k][f'{ManufOrTopC}'] == 'Others', 1, 0)
    modified_manuf_P12M_new[k] = modified_manuf_P12M_new[k].sort_values(['Sorting', "Total Value Share"], ascending = [True, False]).drop(columns = 'Sorting')
    modified_manuf_P12M_new[k] = modified_manuf_P12M_new[k][modified_manuf_P12M_new[k]['Value Share P12M']!=0]

In [110]:
outputPath = os.getcwd()+f"\\Landscape Market(trend-Concent-growth)&ValAvgPrice_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

## Second Landscape Deck: Category Trend - Overview, Share Evolution

In [153]:
index = [8, 9,9,10]
index=[i for i in index if i !=[]]


duplication_7 = [len(modified_calendar_new.keys())]
duplication_8_bymanuf = [len(modified_manuf_evolution_sorted_new.keys())]
duplication_8 = [len(modified_brands_evolution_sorted_new.keys())]
duplication_9 = [len(categories_overview_dfs_new.keys())]

duplication = duplication_7 + duplication_8_bymanuf+duplication_8 + duplication_9
duplication = [item for item in duplication if item !=0]


section_names_slide7 =  ["Category Trends"]
section_names_slide8_bymanuf =  ['Share Evolution By Manuf']
section_names_slide8 =  ['Share Evolution By Brand']
section_names_slide9 =  ["Category Overview"]

section_names = [*section_names_slide7,*section_names_slide8_bymanuf,*section_names_slide8,*section_names_slide9]

path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape duplicate Category(Trend-Overview).pptx'



In [154]:
print(len(index))
# print(index)
print(len(duplication))
# print(duplication)
print(len(section_names))
# print(section_names)

print(sum(duplication))


4
4
4
565


In [ ]:
slideDuplication(index,duplication,section_names,path,new_pre)

KeyboardInterrupt: 

In [177]:
prs = Presentation(new_pre)

def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

#### Slide9_Category_Trend

In [178]:
p = 0
Category_Trend(prs,list(modified_calendar_new.keys()), modified_calendar_new, position = calculate_position(p))
p+=1

#### Slide10_Share_Evolution

In [179]:
for k in modified_brands_evolution_sorted_new.keys():
    t = modified_brands_evolution_sorted_new[k].copy()
    t['Tot Sort'] = np.where(t[f'{BrandOrTopB}'] == 'Total', 0, 1)
    t = t.sort_values(['Year', 'Tot Sort'], ascending=[True, True])
    t = t.drop(columns='Tot Sort')
    modified_brands_evolution_sorted_new[k] = t

for k in modified_manuf_evolution_sorted_new.keys():
    t = modified_manuf_evolution_sorted_new[k].copy()
    t['Tot Sort'] = np.where(t[f'{ManufOrTopC}'] == 'Total', 0, 1)
    t = t.sort_values(['Year', 'Tot Sort'], ascending=[True, True])
    t = t.drop(columns='Tot Sort')
    modified_manuf_evolution_sorted_new[k] = t

In [180]:
Share_Evolution(prs, list(modified_manuf_evolution_sorted_new.keys()), modified_manuf_evolution_sorted_new, position = calculate_position(p),colname=f"{ManufOrTopC}")
p+=1
Share_Evolution(prs, list(modified_brands_evolution_sorted_new.keys()), modified_brands_evolution_sorted_new, position = calculate_position(p),colname=f'{BrandOrTopB}')
p+=1

### Slide11_CategoryOverview

In [181]:
CategoryOverview(prs,modified_categories_dfs_new,col='MonthYear',position=calculate_position(p),scope = sectors, slide_by = 'Sector')
p+=1

In [182]:
outputPath = os.getcwd()+f"\\Landscape Category(Trend-Overview)_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

# Third Landscape Deck (Share and Growth)

In [32]:
categorizations = [
    ("Category", categories),
    ("Sector", sectors),
    ("Segment", segments),
    ("SubSegment", subsegments),
    ("SubCategory", subcategories),

]

# Initialize the updated dictionaries and lists
market_Catg_list = {}
# slide13Dup = []
section_names_slide_13 = []

# Loop through the original markets
for idx, val in enumerate(markets_val):
    if len(val) > 0:
        for cat_name, cat_list in categorizations:
            if len(cat_list) > 0:
                # Modify key and section names to include the categorization level
                key_name = f"{markets_name[idx]}_{cat_name}"
                section_title = f"Share and Growth By {cat_name} for {slide_by[idx]} For {slide_for[idx]}"
                
                # Add the categorized data to the dictionary
                market_Catg_list[key_name] = [val, markets_dfs[idx], client_dfs[idx], slide_by[idx], slide_for[idx], cat_list]
                # slide13Dup.append(len(client_dfs[idx].keys()))
                section_names_slide_13.append(section_title)
                


In [33]:
retailerDup = [len(retailerCatDic),len(retailerSecDic),len(retailerSegDic),len(retailerSubSegDic),len(retailerSubCatDic)]*retMarketLength
channelDup = [len(channelCatDic),len(channelSecDic),len(channelSegDic),len(channelSubSegDic),len(channelSubCatDic)]*chanMarketLength
custDup = [len(custCatDic),len(custSecDic),len(custSegDic),len(custSubSegDic),len(custSubCatDic)]*custMarketLength


In [34]:
slide13Index=[*[12]* (((5 -((not sectors) + (not segments) + (not subsegments)+ (not subcategories)) )) *(9 - ((not regions_RET)+(not channels_RET) + (not market_RET)+ (not regions_CHAN)+ (not channels_CHAN)+ (not market_CHAN)+ (not regions_CUST)+ (not channels_CUST)+ (not market_CUST))  ))]

slide13Dup = retailerDup + channelDup + custDup
slide_12_13_index = [*[11]*(4 -((not sectors) + (not segments) + (not subsegments)+ (not subcategories)) ) ]+slide13Index

duplication = [len(cea_sectors_clients_new.keys()),len(cea_segment_clients_new.keys()),len(cea_subsegment_clients_new.keys()),len(cea_subcategory_clients_new.keys())] + slide13Dup
duplication_12_13 =[ i for i in duplication if i !=0 ]
section_names_slide11 =  (["Sales and Growth By Sector"] if sectors else [])+(["Sales and Growth By Segment"] if len(segments)>0 else [])+(["Sales and Growth By SubSegment"] if len(subsegments)>0 else [])+(["Sales and Growth By SubCategories"] if len(subcategories)>0 else [])



section_names_12_13 = [*section_names_slide11,*section_names_slide_13]

In [35]:
momentum_index=[*[13]* (1+((4 -((not segments) + (not subsegments)+ (not subcategories)) )) *(3 - ((not retailers)+(not channels) + (not cust))  ))]

momentum_index=[i for i in momentum_index if i !=[]]

momentum_dupl_1 = [(len(modified_momentum_retailers.keys())if retailers else 0), (len(modified_momentum_channels.keys())if channels else 0), (len(modified_momentum_cust.keys())if cust else 0)]
momentum_dupl_2 = [(len(modified_momentum_retailers_seg.keys())if retailers else 0), (len(modified_momentum_channels_seg.keys())if channels else 0), (len(modified_momentum_cust_seg.keys())if cust else 0)]if segments else [0]
momentum_dupl_3 = [(len(modified_momentum_retailers_subseg.keys())if retailers else 0), (len(modified_momentum_channels_subseg.keys())if channels else 0), (len(modified_momentum_cust_subseg.keys())if cust else 0)]if subsegments else [0]
momentum_dupl_4 = [(len(modified_momentum_retailers_subcatg.keys())if retailers else 0), (len(modified_momentum_channels_subcatg.keys())if channels else 0), (len(modified_momentum_cust_subcatg.keys())if cust else 0)]if subcategories else [0]
momentum_dupl_5 = [len(modified_momentum_dfs.keys())]


momentum_duplication = momentum_dupl_1 + momentum_dupl_2 + momentum_dupl_3 + momentum_dupl_4 + momentum_dupl_5 
momentum_duplication = [item for item in momentum_duplication if item !=0]

momentum_section_name1 = (['Momentum Analysis By Retailers'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels'] if len(channels)>0 else [] ) + ([ f'Momentum Analysis By {customareas}'] if len(cust)>0 else [])
momentum_section_name2 = ((['Momentum Analysis By Retailers By Segments'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels By Segments'] if len(channels)>0 else [] ) + ([ f'Momentum Analysis By {customareas} By Segments'] if len(cust)>0 else [] ))if segments else []
momentum_section_name3 = ((['Momentum Analysis By Retailers By SubSegments'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels By SubSegments'] if len(channels)>0 else [] ) + ([ f'Momentum Analysis By {customareas} By SubSegments'] if len(cust)>0 else [] ))if subsegments else []
momentum_section_name4 = ((['Momentum Analysis By Retailers By SubCategories'] if len(retailers)>0 else [] ) + ([ 'Momentum Analysis By Channels By SubCategories'] if len(channels)>0 else [] ) + ([ f'Momentum Analysis By {customareas} By SubCategories'] if len(cust)>0 else [] ))if subcategories else []
momentum_section_name5 = "Momentum Analysis By Sectors"

momentum_section_names = [ *momentum_section_name1,*momentum_section_name2,
                  *momentum_section_name3, *momentum_section_name4, momentum_section_name5  
                 ]


In [36]:
index=[
     4,
                retailerREGIndex if regions_RET else [], retailerREGIndex if regions_RET else [], 
                retailerCHAIndex if channels_RET else [], retailerCHAIndex if channels_RET else [], 
                retailerMARIndex if market_RET else [], retailerMARIndex if market_RET else [], 
                channelREGIndex if regions_CHAN else [], channelREGIndex if regions_CHAN else [], 
                channelCHAIndex if channels_CHAN else [], channelCHAIndex if channels_CHAN else [], 
                channelMARIndex if market_CHAN else [], channelMARIndex if market_CHAN else [], 
                custREGIndex if regions_CUST else [], custREGIndex if regions_CUST else [], 
                custCHAIndex if channels_CUST else [], custCHAIndex if channels_CUST else [],  
                custMARIndex if market_CUST else [], custMARIndex if market_CUST else [],           
                sectorIndex if sectors else [],sectorIndex if sectors else [],
                segmentIndex if segments else [],segmentIndex if segments else [],
                subsegmentIndex if subsegments else [],subsegmentIndex if subsegments else [],
                subcategoryIndex if subcategories else [],subcategoryIndex if subcategories else []] + slide_12_13_index + momentum_index
index=[i for i in index if i !=[]]

duplication_5 = [len(modified_manuf_P12M_new.keys())] 
duplication_6area = [1 if regions_RET else 0, 1 if regions_RET else 0,1 if channels_RET else 0, 1 if channels_RET else 0,1 if market_RET else 0, 1 if market_RET else 0,1 if regions_CHAN else 0, 1 if regions_CHAN else 0,1 if channels_CHAN else 0, 1 if channels_CHAN else 0,1 if market_CHAN else 0, 1 if market_CHAN else 0,1 if regions_CUST else 0, 1 if regions_CUST else 0, 1 if channels_CUST else 0, 1 if channels_CUST else 0, 1 if market_CUST else 0, 1 if market_CUST else 0]
duplication_6 = [1 if sectors else 0, 1 if sectors else 0, 1 if segments else 0, 1 if segments else 0, 1 if subsegments else 0, 1 if subsegments else 0, 1 if subcategories else 0, 1 if subcategories else 0]

duplication = duplication_5 + duplication_6area + duplication_6 + duplication_12_13 + momentum_duplication

duplication = [item for item in duplication if item !=0]

section_names_slide5 = ['Share and Growth By Manufacturer/Brands']
section_names_slide6area = (['Share And Growth By Manufacturer By Retailer', 'Share And Growth By Brands By Retailer'] if len(regions_RET)>0 else [] ) +(['Share And Growth By Manufacturer By Retailer', 'Share And Growth By Brands By Retailer'] if len(channels_RET)>0 else [] ) +(['Share And Growth By Manufacturer By Retailer', 'Share And Growth By Brands By Retailer'] if len(market_RET)>0 else [] ) + (['Share And Growth By Manufacturer By Channel', 'Share And Growth By Brands By Channel'] if len(regions_CHAN)>0 else [] ) + (['Share And Growth By Manufacturer By Channel', 'Share And Growth By Brands By Channel'] if len(channels_CHAN)>0 else [] ) + (['Share And Growth By Manufacturer By Channel', 'Share And Growth By Brands By Channel'] if len(market_CHAN)>0 else [] ) + (['Share And Growth By Manufacturer By '+ customareas, 'Share And Growth By Brands By '+ customareas] if len(regions_CUST)>0 else [] ) + (['Share And Growth By Manufacturer By '+ customareas, 'Share And Growth By Brands By '+ customareas] if len(channels_CUST)>0 else [] ) + (['Share And Growth By Manufacturer By '+ customareas, 'Share And Growth By Brands By '+ customareas] if len(market_CUST)>0 else [] )
section_names_slide6 = (['Share And Growth By Manufacturer By Sector', 'Share And Growth By Brands By Sector'] if len(sectors)>0 else [] ) + (['Share And Growth By Manufacturer By Segment', 'Share And Growth By Brands By Segment'] if len(segments)>0 else [] ) + (['Share And Growth By Manufacturer By SubSegment', 'Share And Growth By Brands By SubSegment'] if len(subsegments)>0 else [] )  + (['Share And Growth By Manufacturer By SubCategory', 'Share And Growth By Brands By SubCategory'] if len(subcategories)>0 else [] )


section_names=[*section_names_slide5,*section_names_slide6area,*section_names_slide6] + section_names_12_13 + momentum_section_names
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape duplicate ShareGrowth.pptx'


In [37]:
print(len(index))
# print(index)
print(len(duplication))
# print(duplication)
print(len(section_names))
# print(section_names)

32
32
32


In [38]:
print(sum(duplication))

583


In [39]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [40]:
prs = Presentation(new_pre)

def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

#### Slide5_Share_Growth

In [41]:
for k in modified_brands_P12M_new.keys():
    t=modified_brands_P12M_new[k].copy()
    t= t.rename(columns = { "Value Share P12M":"Value Share", "Share DYA P12M":"Share DYA", "Av Price/KG P12M":"Av Price/KG"})
    modified_brands_P12M_new[k] = t

In [42]:
p = 0
Share_Growth(prs, list(modified_manuf_P12M_new.keys()), modified_manuf_P12M_new, modified_brands_P12M_new, position =calculate_position(p))
p+=1

In [43]:
for k in modified_brands_P12M_new.keys():
    t=modified_brands_P12M_new[k].copy()
    t= t.rename(columns = {"Value Share": "Value Share P12M", "Share DYA":"Share DYA P12M", "Av Price/KG":"Av Price/KG P12M"})
    modified_brands_P12M_new[k] = t[t['Value Share P12M'] != 0]


### Slide6_Share and Growth By Manufacturer 

In [44]:
c = calculate_position(p)
if len(regions_RET)!=0:
    for key in shareGrowthDf_retailers_REG.keys():
        retailers_REG_list = [shareGrowthDf_retailers_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_REG[key]), 3)]
        for sublist in retailers_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Retailer', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_retailers_REG.keys():
        retailers_REG_list = [shareGrowthDf_retailers_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_REG[key]), 3)]
        for sublist in retailers_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Retailer', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1
    p+=1 
if len(channels_RET)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_retailers_CHAN.keys():
        retailers_CHAN_list = [shareGrowthDf_retailers_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_CHAN[key]), 3)]
        for sublist in retailers_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Retailer', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_retailers_CHAN.keys():
        retailers_CHAN_list = [shareGrowthDf_retailers_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_CHAN[key]), 3)]
        for sublist in retailers_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Retailer', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(market_RET)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_retailers_MAR.keys():
        retailers_MAR_list = [shareGrowthDf_retailers_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_MAR[key]), 3)]
        for sublist in retailers_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Retailer', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_retailers_MAR.keys():
        retailers_MAR_list = [shareGrowthDf_retailers_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_retailers_MAR[key]), 3)]
        for sublist in retailers_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Retailer', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1 
    p+=1
    

In [45]:
c=calculate_position(p)  
if len(regions_CHAN)!=0:
    for key in shareGrowthDf_channels_REG.keys():
        channels_REG_list = [shareGrowthDf_channels_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_REG[key]), 3)]
        for sublist in channels_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Channel', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_channels_REG.keys():
        channels_REG_list = [shareGrowthDf_channels_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_REG[key]), 3)]
        for sublist in channels_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Channel', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1
    p+=1   
if len(channels_CHAN)!=0:  
    c=calculate_position(p) 
    for key in shareGrowthDf_channels_CHAN.keys():
        channels_CHAN_list = [shareGrowthDf_channels_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_CHAN[key]), 3)]
        for sublist in channels_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Channel', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_channels_CHAN.keys():
        channels_CHAN_list = [shareGrowthDf_channels_CHAN[key][i:i+3] for i in range(0,len(shareGrowthDf_channels_CHAN[key]), 3)]
        for sublist in channels_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Channel', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(market_CHAN)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_channels_MAR.keys():
        channels_MAR_list = [shareGrowthDf_channels_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_MAR[key]), 3)]
        for sublist in channels_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Channel', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_channels_MAR.keys():
        channels_MAR_list = [shareGrowthDf_channels_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_channels_MAR[key]), 3)]
        for sublist in channels_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Channel', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1 
    p+=1

In [46]:
c=calculate_position(p)  
if len(regions_CUST)!=0:
    for key in shareGrowthDf_custom_REG.keys():
        custom_REG_list = [shareGrowthDf_custom_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_REG[key]), 3)]
        for sublist in custom_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By '+ customareas, list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_custom_REG.keys():
        custom_REG_list = [shareGrowthDf_custom_REG[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_REG[key]), 3)]
        for sublist in custom_REG_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By '+ customareas, list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1
    p+=1     
if len(channels_CUST)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_custom_CHAN.keys():
        custom_CHAN_list = [shareGrowthDf_custom_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_CHAN[key]), 3)]
        for sublist in custom_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By '+ customareas, list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_custom_CHAN.keys():
        custom_CHAN_list = [shareGrowthDf_custom_CHAN[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_CHAN[key]), 3)]
        for sublist in custom_CHAN_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By '+ customareas, list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(market_CUST)!=0:
    c=calculate_position(p) 
    for key in shareGrowthDf_custom_MAR.keys():
        custom_MAR_list = [shareGrowthDf_custom_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_MAR[key]), 3)]
        for sublist in custom_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By '+ customareas, list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    c=calculate_position(p)
    for key in shareGrowthDf_custom_MAR.keys():
        custom_MAR_list = [shareGrowthDf_custom_MAR[key][i:i+3] for i in range(0, len(shareGrowthDf_custom_MAR[key]), 3)]
        for sublist in custom_MAR_list:
            Share_Growth_Separately_area( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By '+ customareas, list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1 
    p+=1

In [47]:
c = calculate_position(p)
if len(sectors)!=0:
    for key in sharGrowthDf_sec.keys():
        sectors_list = [sharGrowthDf_sec[key][i:i+3] for i in range(0, len(sharGrowthDf_sec[key]), 3)]
        print(sectors_list)
        for sublist in sectors_list:
            print(sublist)
            Share_Growth_Separately( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Sector', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1

    c=calculate_position(p)
    for key in sharGrowthDf_sec.keys():
        sectors_list = [sharGrowthDf_sec[key][i:i+3] for i in range(0, len(sharGrowthDf_sec[key]), 3)]
        for sublist in sectors_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Sector', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(segments)!=0:
    c = calculate_position(p)
    for key in sharGrowthDf_seg.keys():
        segments_list = [sharGrowthDf_seg[key][i:i+3] for i in range(0, len(sharGrowthDf_seg[key]), 3)]
        for sublist in segments_list:
            Share_Growth_Separately( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By Segment', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  
    p+=1
    
    c=calculate_position(p)
    for key in sharGrowthDf_seg.keys():
        segments_list = [sharGrowthDf_seg[key][i:i+3] for i in range(0, len(sharGrowthDf_seg[key]), 3)]
        for sublist in segments_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By Segment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1

if len(subsegments)!=0:
    c = calculate_position(p)
    for key in sharGrowthDf_subseg.keys():
        subsegments_list = [sharGrowthDf_subseg[key][i:i+3] for i in range(0, len(sharGrowthDf_subseg[key]), 3)]
        for sublist in subsegments_list:
            Share_Growth_Separately( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By SubSegment', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  # Correctly incrementing the variable c
    p+=1
    
    c=calculate_position(p)
    for key in sharGrowthDf_subseg.keys():
        subsegments_list = [sharGrowthDf_subseg[key][i:i+3] for i in range(0, len(sharGrowthDf_subseg[key]), 3)]
        for sublist in subsegments_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By SubSegment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1


if len(subcategories)!=0:
    c = calculate_position(p)
    for key in sharGrowthDf_subcat.keys():
        subcateg_list = [sharGrowthDf_subcat[key][i:i+3] for i in range(0, len(sharGrowthDf_subcat[key]), 3)]
        for sublist in subcateg_list:
            Share_Growth_Separately( prs,[key], modified_manuf_P12M_new,position=c,slide_by='Manufacturer | By SubCategory', list_tables=sublist,first_col=f'{ManufOrTopC}' )
            c += 1  # Correctly incrementing the variable c
    p+=1
    
    c=calculate_position(p)
    for key in sharGrowthDf_subcat.keys():
        subcateg_list = [sharGrowthDf_subcat[key][i:i+3] for i in range(0, len(sharGrowthDf_subcat[key]), 3)]
        for sublist in subcateg_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_new,position=c,slide_by='Brands | By SubCategory', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1            

[['Dermaplane', 'Disposable', 'System']]
['Dermaplane', 'Disposable', 'System']
[['Disposable', 'System']]
['Disposable', 'System']
[['Dermaplane', 'Disposable', 'System']]
['Dermaplane', 'Disposable', 'System']
[['Dermaplane', 'Disposable', 'System']]
['Dermaplane', 'Disposable', 'System']
[['Disposable', 'System']]
['Disposable', 'System']
[['Dermaplane', 'Disposable', 'System']]
['Dermaplane', 'Disposable', 'System']
[['Dermaplane', 'Disposable', 'System']]
['Dermaplane', 'Disposable', 'System']
[['Dermaplane', 'Disposable', 'System']]
['Dermaplane', 'Disposable', 'System']
[['Dermaplane', 'Disposable', 'System']]
['Dermaplane', 'Disposable', 'System']
[['Disposable', 'System']]
['Disposable', 'System']


## Slide 12 : Category Evolution Analysis 

In [48]:
manuf_brand_list = client_manuf + client_brands

In [49]:
if sectors:
    categoryEvolution(prs,cea_sectors_clients_new,cea_sectors_P12M_new,cea_dya_sectors_new,cea_sectors_clients_total_new,manuf_brand_list,col="Sector",position = calculate_position(p))
    p+=1

In [50]:
# if len(segments)!=0:
#     posadd=calculate_position_12_13(p)
#     for sec in sectors:
#         categoryEvolution(prs,cea_segment_clients_new,cea_segment_P12M_new,cea_dya_segment_new,cea_segment_clients_total_new,manuf_brand_list,col='Segment',position = posadd)
#         posadd += len(cea_segment_clients_new.keys())
#     p+=1

if len(segments)!=0:
    posadd=calculate_position(p)
    categoryEvolution(prs,cea_segment_clients_new,cea_segment_P12M_new,cea_dya_segment_new,cea_segment_clients_total_new,manuf_brand_list,col='Segment',position = posadd)
    p+=1    

In [51]:
if len(subsegments)!=0:
    posadd=calculate_position(p)
    categoryEvolution(prs,cea_subsegment_clients_new,cea_subsegment_P12M_new,cea_dya_subsegment_new,cea_subsegment_clients_total_new,manuf_brand_list,col='SubSegment',position = posadd)
    p+=1


In [52]:
if len(subcategories)!=0:
    posadd=calculate_position(p)
    categoryEvolution(prs,cea_subcategory_clients_new,cea_subcategory_P12M_new,cea_dya_subcategory_new,cea_subcategory_clients_total_new,manuf_brand_list,col='SubCategory',position = posadd)   
    p+=1  

## Slide 13 : Share And Growth

In [53]:
catSecSeg = (
    categories + sectors + segments + subsegments + subcategories
    if any([categories, sectors, segments, subsegments, subcategories])
    else []
)

###### Retailer

In [54]:
# if retailers_dfs_new:
#     for key,val in retailerField.items():
#         if val[0]:

#             if categories:
#                 clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerCatDic}
#                 totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in categories}
#                 shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position_12_13(p)) 
#             p+=1

#             if sectors:   
#                 clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSecDic}
#                 totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in sectors}
#                 shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position_12_13(p)) 
#             p+=1


#             if segments: 
#                 clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSegDic}
#                 totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in segments}
#                 shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position_12_13(p)) 
#             p+=1


#             if subsegments:
#                 clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSubSegDic}
#                 totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subsegments}
#                 shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position_12_13(p)) 
#             p+=1

            
#             if subcategories:
#                 clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSubCatDic}
#                 totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subcategories}

#                 shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position_12_13(p)) 


if retailers_dfs_new:
    for key,val in retailerField.items():
        if val[0]:
 
            if categories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in categories}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if sectors:  
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSecDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in sectors}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if segments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in segments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if subsegments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSubSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subsegments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
           
            if subcategories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in retailerSubCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subcategories}
 
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1

###### Channel

In [55]:
if channel_dfs_new:
    for key,val in channelField.items():
        if val[0]:
            if categories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in categories}
               
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
           
 
            if sectors:  
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSecDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in sectors}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if segments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in segments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if subsegments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSubSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subsegments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if subcategories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1].keys() if key in channelSubCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0].keys() if key in subcategories}
 
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1

###### Custom

In [56]:
if cust_dfs_new:
    for key,val in custField.items():
        if val[0]:
 
            if categories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in categories}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if sectors:  
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSecDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in sectors}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
 
            if segments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in segments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
            if subsegments:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSubSegDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in subsegments}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1
 
           
            if subcategories:
                clientDic = {key : pd.concat([val[3][key],val[1][key]]) for key in val[1] if key in custSubCatDic}
                totalDic = {key : pd.concat([val[2][key],val[0][key]]) for key in val[0] if key in subcategories}
                shareGrowth(prs,clientDic,totalDic,catSecSeg,col=key,position = calculate_position(p))
                p+=1

#### Momentum Analysis

In [57]:

if retailers:
    Momentum_Analysis(prs, list(modified_momentum_retailers.keys()), modified_momentum_retailers, position = calculate_position(p)) 
    p+=1

if channels:
    Momentum_Analysis(prs, list(modified_momentum_channels.keys()), modified_momentum_channels, position = calculate_position(p)) 
    p+=1

if cust:
    Momentum_Analysis(prs, list(modified_momentum_cust.keys()), modified_momentum_cust, position = calculate_position(p)) 
    p+=1


if segments:
    if retailers:
        Momentum_Analysis(prs, list(modified_momentum_retailers_seg.keys()), modified_momentum_retailers_seg, position = calculate_position(p)) 
        p+=1

    if channels:
        Momentum_Analysis(prs, list(modified_momentum_channels_seg.keys()), modified_momentum_channels_seg, position = calculate_position(p)) 
        p+=1

    if cust:
        Momentum_Analysis(prs, list(modified_momentum_cust_seg.keys()), modified_momentum_cust_seg, position = calculate_position(p)) 
        p+=1

if subsegments:
    if retailers:
        Momentum_Analysis(prs, list(modified_momentum_retailers_subseg.keys()), modified_momentum_retailers_subseg, position = calculate_position(p)) 
        p+=1

    if channels:
        Momentum_Analysis(prs, list(modified_momentum_channels_subseg.keys()), modified_momentum_channels_subseg, position = calculate_position(p)) 
        p+=1

    if cust:
        Momentum_Analysis(prs, list(modified_momentum_cust_subseg.keys()), modified_momentum_cust_subseg, position = calculate_position(p)) 
        p+=1

if subcategories:
    if retailers:
        Momentum_Analysis(prs, list(modified_momentum_retailers_subcatg.keys()), modified_momentum_retailers_subcatg, position = calculate_position(p)) 
        p+=1

    if channels:
        Momentum_Analysis(prs, list(modified_momentum_channels_subcatg.keys()), modified_momentum_channels_subcatg, position = calculate_position(p)) 
        p+=1

    if cust:
        Momentum_Analysis(prs, list(modified_momentum_cust_subcatg.keys()), modified_momentum_cust_subcatg, position = calculate_position(p)) 
        p+=1


Momentum_Analysis(prs, list(modified_momentum_dfs.keys()), modified_momentum_dfs, position = calculate_position(p))

In [58]:
outputPath = os.getcwd()+f"\\Landscape Pre-Output ShareGrowth_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

In [190]:
output_path_final = os.getcwd()+f"\\Landscape ShareGrowth_{datetime.today().strftime("%d-%m")}.pptx"
open_chart_data_in_excel(outputPath,output_path_final)

Slide 778: Opened Excel workbook: Book1
Slide 779: Opened Excel workbook: Book1
Slide 780: Opened Excel workbook: Book1
Slide 781: Opened Excel workbook: Book1
Slide 782: Opened Excel workbook: Book1
Slide 783: Opened Excel workbook: Book1
Slide 784: Opened Excel workbook: Book1
Slide 785: Opened Excel workbook: Book1
Slide 786: Opened Excel workbook: Book1
Slide 787: Opened Excel workbook: Book1
Slide 788: Opened Excel workbook: Book1
Slide 789: Opened Excel workbook: Book1
Slide 790: Opened Excel workbook: Book1
Slide 791: Opened Excel workbook: Book1
Slide 792: Opened Excel workbook: Book1
Slide 793: Opened Excel workbook: Book1
Slide 794: Opened Excel workbook: Book1
Slide 795: Opened Excel workbook: Book1
Slide 796: Opened Excel workbook: Book1
Slide 797: Opened Excel workbook: Book1
Slide 798: Opened Excel workbook: Book1
Slide 799: Opened Excel workbook: Book1
Slide 800: Opened Excel workbook: Book1
Slide 801: Opened Excel workbook: Book1
Slide 802: Opened Excel workbook: Book1


# RPVM

In [7]:
modified_revenue_PVM_Manuf = {}

for key in revenue_PVM_manuf.keys():
    df = revenue_PVM_manuf[key].copy()
    df = DetectHeader(df)
    df[f"{ManufOrTopC}"] = df[f"{ManufOrTopC}"].ffill()
    df['Value Share'] = df['Value Share'].astype(float) 
    df['Revenue By PVM'] = df['Revenue By PVM'].astype(float) / 1000

    # Get only the client brands that are present in the current df
    relevant_client_brands = [brand for brand in client_manuf if brand in df[f"{ManufOrTopC}"].values]

    # Proceed only if there are relevant client brands in the df
    if relevant_client_brands:
        filtered_df = df[
            (df[f"{ManufOrTopC}"].str.contains(" Total")) & 
            (df[f"{ManufOrTopC}"] != "Grand Total") &
            (df[f"{ManufOrTopC}"] != "All Others Total") &  
            (~df[f"{ManufOrTopC}"].str.replace(" Total", "").isin(client_manuf))
        ].nlargest(1, "Value Share")

        # Check if the filtered DataFrame is not empty
        if not filtered_df.empty:
            top_competitor = filtered_df[f"{ManufOrTopC}"].str.replace(" Total", "").iloc[0]
        else:
            top_competitor = None  

        if top_competitor:
            # Use only the relevant client brands
            for brand in relevant_client_brands:
                filtered_df = df[(df[f"{ManufOrTopC}"] == brand) | (df[f"{ManufOrTopC}"] == top_competitor)]
                new_key = f"{key} | {brand}"
                modified_revenue_PVM_Manuf[new_key] = filtered_df

In [8]:
modified_revenue_PVM_Brand = {}

for key in revenue_PVM_brand.keys():
    df = revenue_PVM_brand[key].copy()
    df = DetectHeader(df)
    df[f'{BrandOrTopB}'] = df[f'{BrandOrTopB}'].ffill()
    df['Value Share'] = df['Value Share'].astype(float)
    df['Revenue By PVM'] = df['Revenue By PVM'].astype(float) / 1000

    # Get only the client brands that are present in the current df
    relevant_client_brands = [brand for brand in client_brands if brand in df[f'{BrandOrTopB}'].values]

    # Proceed only if there are relevant client brands in the df
    if relevant_client_brands:
        filtered_df = df[
            (df[f"{BrandOrTopB}"].str.contains(" Total")) & 
            (df[f"{BrandOrTopB}"] != "Grand Total") &
            (df[f"{BrandOrTopB}"] != "All Others Total") &  
            (~df[f"{BrandOrTopB}"].str.replace(" Total", "").isin(client_brands))
        ].nlargest(1, "Value Share")

        # Check if the filtered DataFrame is not empty
        if not filtered_df.empty:
            top_competitor = filtered_df[f"{BrandOrTopB}"].str.replace(" Total", "").iloc[0]
        else:
            top_competitor = None  

        if top_competitor:
            # Use only the relevant client brands
            for brand in relevant_client_brands:
                filtered_df = df[(df[f'{BrandOrTopB}'] == brand) | (df[f'{BrandOrTopB}'] == top_competitor)]
                new_key = f"{key} | {brand}"
                modified_revenue_PVM_Brand[new_key] = filtered_df

In [11]:
index = [*[14]*(1)]
index=[i for i in index if i !=[]]

duplication_1 = [len(modified_revenue_PVM_Manuf.keys()) +len(modified_revenue_PVM_Brand.keys())]
# duplication_2=[len(modified_revenue_PVM_Brand.keys())]

duplication = duplication_1


duplication = [item for item in duplication if item !=0]

section_names_slide1 = ["Revenue PVM By Manufacturer & Brands"]
# section_names_slide2 = ["Revenue PVM By Brands"]

section_names = [ *section_names_slide1]


In [12]:
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape Duplicate RPVM.pptx'

In [13]:
print(len(index))
# print(index)
print(len(duplication))
# print(duplication)
print(len(section_names))
# print(section_names)
print(sum(duplication))

1
1
1
578


In [14]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [15]:
def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))


ppt_app = win32com.client.Dispatch("PowerPoint.Application")

prs = ppt_app.Presentations.open(new_pre)


In [18]:
# Combine the keys from both dictionaries into one list
combined_list_duplicates = list(modified_revenue_PVM_Manuf.keys()) + list(modified_revenue_PVM_Brand.keys())

# Combine the dictionaries into one, with Brand entries added after Manuf entries
combined_modified_RPVM = {**modified_revenue_PVM_Manuf, **modified_revenue_PVM_Brand}


# Run the RPVM function with the combined list and dictionary
RPVM(prs, combined_list_duplicates, combined_modified_RPVM, position=calculate_position(0))


Error on slide 577, retry 1: (-2147352567, 'Exception occurred.', (0, 'Microsoft PowerPoint', "View.GotoSlide : Integer out of range. 579 is not in Index's valid range of 1 to 578.", '', 0, -2147188160), None)
Error on slide 577, retry 2: (-2147352567, 'Exception occurred.', (0, 'Microsoft PowerPoint', "View.GotoSlide : Integer out of range. 579 is not in Index's valid range of 1 to 578.", '', 0, -2147188160), None)
Error on slide 577, retry 3: (-2147352567, 'Exception occurred.', (0, 'Microsoft PowerPoint', "View.GotoSlide : Integer out of range. 579 is not in Index's valid range of 1 to 578.", '', 0, -2147188160), None)
Error on slide 577, retry 4: (-2147352567, 'Exception occurred.', (0, 'Microsoft PowerPoint', "View.GotoSlide : Integer out of range. 579 is not in Index's valid range of 1 to 578.", '', 0, -2147188160), None)
Error on slide 577, retry 5: (-2147352567, 'Exception occurred.', (0, 'Microsoft PowerPoint', "View.GotoSlide : Integer out of range. 579 is not in Index's vali

In [19]:

output_path = os.path.join(os.getcwd(), 'Landscape RPVM.pptx')
prs.SaveAs(output_path)


# Slides WITHOUT forcing CLIENT BRAND

In [82]:
if not Sections_WO_CB:
    print("Stopping the script as 'Sections_WO_CB' is set to False.")
    sys.exit()  # Terminate the script here


Stopping the script as 'Sections_WO_CB' is set to False.


SystemExit: 

In [ ]:
modified_brands_share_noCB = {}
modified_brands_totals_noCB = {}
modified_brands_P12M_noCB = {}


for s in brands_dfs_new.keys():
    t = brands_dfs_new[s].copy()
    t=DetectHeader(t).fillna(0)
    t.columns = t.columns.str.strip()
   
    t = t.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
    t = t.applymap(lambda x: 0 if pd.isna(x) else x)
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    if not mod.empty:
        modified_brands_share_noCB[s] = mod
    if not tot.empty:
        modified_brands_totals_noCB[s] = tot


df_toDrop =[key for key in modified_brands_share_noCB.keys() if (modified_brands_share_noCB[key].shape[1]>2) and (modified_brands_share_noCB[key].shape[1]<4)]
for key in df_toDrop:
    del modified_brands_share_noCB[key]

    
emptyDf=[key for key in modified_brands_share_noCB.keys() if modified_brands_share_noCB[key].shape[1]<4]
for dic in [modified_brands_share_noCB]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            print()
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0
p = 10 if len(client_brands) < 10 else len(client_brands) + 1

modified_brands_share_noCB = dfSort(modified_brands_share_noCB, [],f'{BrandOrTopB}',p, 'Total Value Share')
for key in modified_brands_share_noCB.keys():
    modified_brands_share_noCB[key] = modified_brands_share_noCB[key][modified_brands_share_noCB[key]['Total Value Share'] != 0]
addOther(modified_brands_share_noCB,col=f'{BrandOrTopB}',TorB="Top")


for s in brands_P12M_dfs_new.keys():
    t = brands_P12M_dfs_new[s].copy()

    t=DetectHeader(t).fillna(0)
    t.columns = t.columns.str.strip()


    t = t.applymap(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == x else x)
    t = t.applymap(lambda x: 0 if pd.isna(x) else x)
    mod = t[(~t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    mod = mod.sort_values([col for col in mod.columns if 'Value Share' in col], ascending=False)
    tot = t[(t[t.columns[0]].astype(str).str.contains('Grand Total'))]
    modified_brands_P12M_noCB[s] = mod



df_toDrop =[key for key in modified_brands_P12M_noCB.keys() if (modified_brands_P12M_noCB[key].shape[1]>2) and (modified_brands_P12M_noCB[key].shape[1]<4)]
for key in df_toDrop:
    del modified_brands_P12M_noCB[key]

emptyDf=[key for key in modified_brands_P12M_noCB.keys() if modified_brands_P12M_noCB[key].shape[1]<4]
for dic in [modified_brands_P12M_noCB]:
    if emptyDf:
        columnsToAdd=list(set(dic.keys())-set(emptyDf))[0]
        for df in emptyDf:
            dic[df]=dic[df].drop(columns=[''])#list(set(dic[df].columns)-set(['']))
            emptyCol=list(set(dic[columnsToAdd].columns)-set(dic[df].columns)-set(['']))
            dic[df][emptyCol]=0.0

modified_brands_P12M_noCB = dfSort(modified_brands_P12M_noCB, [],f'{BrandOrTopB}',10, 'Value Share')


In [ ]:
index = [
    *[0]*(1),
           *[1]*(1),
                4,
                sectorIndex if sectors else [],
                segmentIndex if segments else [],
                subsegmentIndex if subsegments else [],
                subcategoryIndex if subcategories else []
                ]
index=[i for i in index if i !=[]]

duplication_1 = [len(modified_brands_share_noCB.keys())]
duplication_2 = [len(modified_brands_share_noCB.keys())]

duplication_5 = [len(modified_manuf_P12M_new.keys())] 
duplication_6 = [1 if sectors else 0, 1 if segments else 0,  1 if subsegments else 0,  1 if subcategories else 0]

duplication = duplication_1 + duplication_2  + duplication_5 + duplication_6


duplication = [item for item in duplication if item !=0]

section_names_slide1 = ["Market Trends by Brands"]
section_names_slide2 = ["Market Concentration By Brands"]
section_names_slide5 = ['Share and Growth By Manufacturer/Brands']
section_names_slide6 = ([ 'Share And Growth By Brands By Sector'] if len(sectors)>0 else [] ) + (['Share And Growth By Brands By Segment'] if len(segments)>0 else [] ) + (['Share And Growth By Brands By SubSegment'] if len(subsegments)>0 else [] )  + (['Share And Growth By Brands By SubCategory'] if len(subcategories)>0 else [] )

section_names = [ *section_names_slide1,*section_names_slide2,
                  *section_names_slide5, *section_names_slide6  
                 ]


In [ ]:
path = os.getcwd() + '\Landscape base.pptx'
new_pre = os.getcwd() + '\Landscape Duplicate Without CB.pptx'

In [ ]:
print(len(index))
# print(index)
print(len(duplication))
print(duplication)
print(len(section_names))
# print(section_names)
print(sum(duplication))

In [ ]:
# slideDuplication(index,duplication,section_names,path,new_pre)


In [ ]:
def calculate_position(end):
    return sum(duplication[i] * (1 if isinstance(index[i], int) else len(index[i])) for i in range(end))

In [ ]:
prs = Presentation(new_pre)


In [ ]:
p = 0
Market_Trends(prs, list(modified_brands_share_noCB.keys()), modified_brands_share_noCB, modified_brands_totals_noCB, client_brands ,position =calculate_position(p), slide_by = f'{BrandOrTopB}', years=years)

p+=1

In [ ]:
Market_Concentration(prs,list(modified_brands_share_noCB.keys()), modified_brands_share_noCB, modified_brands_totals_noCB, client_brands ,position =  calculate_position(p), slide_by = f'{BrandOrTopB}',years=years)

p+=1

In [ ]:
for k in modified_brands_P12M_noCB.keys():
    modified_brands_P12M_noCB[k] = modified_brands_P12M_noCB[k].rename(columns = { "Value Share P12M":"Value Share", "Share DYA P12M":"Share DYA", "Av Price/KG P12M":"Av Price/KG"})
    modified_brands_P12M_noCB[k] = modified_brands_P12M_noCB[k].sort_values('Value Share', ascending = False)
    modified_brands_P12M_noCB[k] = modified_brands_P12M_noCB[k][modified_brands_P12M_noCB[k]['Value Share']!=0]    
    modified_brands_P12M_noCB[k]['Sorting'] = np.where(modified_brands_P12M_noCB[k][f'{BrandOrTopB}'] == 'Others', 1, 0)
    modified_brands_P12M_noCB[k] = modified_brands_P12M_noCB[k].sort_values(['Sorting',"Value Share"], ascending = [True, False]).drop(columns = 'Sorting')


In [ ]:
Share_Growth(prs, list(modified_manuf_P12M_new.keys()), modified_manuf_P12M_new, modified_brands_P12M_noCB, position =calculate_position(p))

p+=1

In [ ]:
for k in modified_brands_P12M_noCB.keys():
    t=modified_brands_P12M_noCB[k].copy()
    t= t.rename(columns = {"Value Share": "Value Share P12M", "Share DYA":"Share DYA P12M", "Av Price/KG":"Av Price/KG P12M"})
    modified_brands_P12M_noCB[k] = t[t['Value Share P12M'] != 0]

In [ ]:
c = calculate_position(p)
if len(sectors)!=0:

    c=calculate_position(p)
    for key in sharGrowthDf_sec.keys():
        sectors_list = [sharGrowthDf_sec[key][i:i+3] for i in range(0, len(sharGrowthDf_sec[key]), 3)]
        for sublist in sectors_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_noCB,position=c,slide_by='Brands | By Sector', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  
    p+=1
if len(segments)!=0:
    
    c=calculate_position(p)
    for key in sharGrowthDf_seg.keys():
        segments_list = [sharGrowthDf_seg[key][i:i+3] for i in range(0, len(sharGrowthDf_seg[key]), 3)]
        for sublist in segments_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_noCB,position=c,slide_by='Brands | By Segment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1

if len(subsegments)!=0:
    
    c=calculate_position(p)
    for key in sharGrowthDf_subseg.keys():
        subsegments_list = [sharGrowthDf_subseg[key][i:i+3] for i in range(0, len(sharGrowthDf_subseg[key]), 3)]
        for sublist in subsegments_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_noCB,position=c,slide_by='Brands | By SubSegment', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1


if len(subcategories)!=0:
    
    c=calculate_position(p)
    for key in sharGrowthDf_subcat.keys():
        subcateg_list = [sharGrowthDf_subcat[key][i:i+3] for i in range(0, len(sharGrowthDf_subcat[key]), 3)]
        for sublist in subcateg_list:
            Share_Growth_Separately( prs,[key], modified_brands_P12M_noCB,position=c,slide_by='Brands | By SubCategory', list_tables=sublist,first_col=f'{BrandOrTopB}' )
            c += 1  # Correctly incrementing the variable c
    p+=1        

In [ ]:
outputPath = os.getcwd()+f"\\Landscape Output WO CB_{datetime.today().strftime("%d-%m")}.pptx"
prs.save(outputPath)

In [ ]:
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Script started at: {time.ctime(start_time)}")
print(f"Script ended at: {time.ctime(end_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")